I recently went through the Deeplearning.ai hugging face course and really liked reading through the sentence embedding module. In finding similarities amoung sentences, I wondered how similar the different models (OpenAI, Mistral, and Anthropic) are. This notebook is an exploration given the same inputs, how similar are the responses from each model.

In [ ]:
from transformers.utils import logging
logging.set_verbosity_error()

In [ ]:
 !pip install sentence-transformers


# Install and set up LLM providers

In [ ]:
# install from PyPI
!pip install anthropic

In [ ]:
!pip install --upgrade openai

In [ ]:
!pip install mistralai

Set up OpenAI API call

In [ ]:
from openai import OpenAI
from google.colab import userdata

openai_client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY'),
)

completion = openai_client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message.content)

Set up Anthropic API call

In [ ]:
import anthropic
from google.colab import userdata

anthropic_client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=userdata.get('ANTHROPIC_API_KEY'),
)
message = anthropic_client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude. Tell me a short story!"}
    ]
)

In [ ]:
print(message.content[0].text)

Set up Mistral API Call

In [ ]:
from google.colab import userdata

from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

model = "mistral-tiny"

client = MistralClient(userdata.get('MISTRAL_API_KEY'))

mistral_chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content="What is the best French cheese?")],
)
print(mistral_chat_response.choices[0].message.content)

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate Text from providers

Generate responses from the 3 different services based on the same prompt.

In [ ]:
prompt = 'I am a developer and I want to learn how to code. My speciality is in backend development and I want to get better at front end. Where do I start?'

In [ ]:
mistral_chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt)],
)

In [ ]:
anthropic_message = anthropic_client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

In [ ]:
openai_completion = openai_client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": prompt}
  ]
)

Get the vector calculations of the messages.

In [ ]:
embeddings_mistral = model.encode(mistral_chat_response.choices[0].message.content, convert_to_tensor=True)

In [ ]:
embeddings_anthropic = model.encode(anthropic_message.content[0].text, convert_to_tensor=True)

In [ ]:
embeddings_openai = model.encode(openai_completion.choices[0].message.content, convert_to_tensor=True)

Calculate cosine similarity among the three different providers

In [ ]:
from sentence_transformers import util

In [ ]:
cosine_scores_mistral_anthropic = util.cos_sim(embeddings_mistral, embeddings_anthropic)

In [ ]:
print(cosine_scores_mistral_anthropic)

In [ ]:
cosine_scores_mistral_openai = util.cos_sim(embeddings_mistral, embeddings_openai)
print(cosine_scores_mistral_openai)

In [ ]:
cosine_scores_openai_anthropic = util.cos_sim(embeddings_openai, embeddings_anthropic)

In [ ]:
print(cosine_scores_openai_anthropic)